In [ ]:
!pip install numpy
!pip install matplotlib
# !pip install opencv-python

!pip uninstall opencv-contrib-python
!pip uninstall opencv-python 

!pip install opencv-python==3.4.2.17 opencv-contrib-python==3.4.2.17
# !pip install opencv-python==3.3.0.10 opencv-contrib-python==3.3.0.10

Found existing installation: opencv-contrib-python 3.4.2.17
Uninstalling opencv-contrib-python-3.4.2.17:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/cv2/*
    /usr/local/lib/python3.7/dist-packages/opencv_contrib_python-3.4.2.17.dist-info/*
Proceed (y/n)? y
  Successfully uninstalled opencv-contrib-python-3.4.2.17
Found existing installation: opencv-python 3.4.2.17
Uninstalling opencv-python-3.4.2.17:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/opencv_python-3.4.2.17.dist-info/*
Proceed (y/n)? y
  Successfully uninstalled opencv-python-3.4.2.17
  Using cached opencv_python-3.4.2.17-cp37-cp37m-manylinux1_x86_64.whl (25.0 MB)
  Using cached opencv_contrib_python-3.4.2.17-cp37-cp37m-manylinux1_x86_64.whl (30.6 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0

In [ ]:
import cv2 as cv
print(cv.__version__)
import sys
from google.colab.patches import cv2_imshow
import numpy as np
from matplotlib import pyplot as plt

3.4.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/computer_vision/data')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


  Using cached opencv_python-3.4.2.17-cp37-cp37m-manylinux1_x86_64.whl (25.0 MB)
  Using cached opencv_contrib_python-3.4.2.17-cp37-cp37m-manylinux1_x86_64.whl (30.6 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


# SIFT

In [ ]:
# Introduction to SIFT (Scale-Invariant Feature Transform)

# harris 모서리는 회전에 불변하나 규모에는 변함 ; 규모 공간 극한 탐지, 키포인트 localization, orientation 지정, 키포인트 descriptor, 키포인트 matching


img = cv.imread('home.jpg')
gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
sift = cv.SIFT_create()
kp = sift.detect(gray,None)
img=cv.drawKeypoints(gray,kp,img)
cv2_imshow(img)
cv.imwrite('sift_keypoints.jpg',img)

AttributeError: ignored

In [ ]:
# coding: utf-8
import warnings
warnings.filterwarnings("ignore")  #忽略警告
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from PIL import Image




def convolve(filter,mat,padding,strides):

    result = None
    filter_size = filter.shape
    mat_size = mat.shape
    if len(filter_size) == 2:
        if len(mat_size) == 3: # 컬러 이미지인 경우
            channel = []
            for i in range(mat_size[-1]):
                pad_mat = np.pad(mat[:,:,i], ((padding[0], padding[1]), (padding[2], padding[3])), 'constant')
                temp = []
                for j in range(0,mat_size[0],strides[1]):
                    temp.append([])
                    for k in range(0,mat_size[1],strides[0]):
                        val = (filter*pad_mat[j:j+filter_size[0],k:k+filter_size[1]]).sum()
                        temp[-1].append(val)
                channel.append(np.array(temp))

            channel = tuple(channel)
            result = np.dstack(channel)
        elif len(mat_size) == 2: # 흑백 이미지인 경우
            channel = []
            pad_mat = np.pad(mat, ((padding[0], padding[1]), (padding[2], padding[3])), 'constant') # 필터 적용을 위한 가장자리 패딩
            for j in range(0, mat_size[0], strides[1]): # stride에 따라 필터 옮겨가며 특징 추출
                channel.append([])
                for k in range(0, mat_size[1], strides[0]):
                    val = (filter * pad_mat[j:j + filter_size[0],k:k + filter_size[1]]).sum()
                    channel[-1].append(val)


            result = np.array(channel)


    return result



def downsample(img,step = 2):
    return img[::step,::step] # 이미지 사이즈 줄이기

def GuassianKernel(sigma , dim):
    '''
    :param sigma: Standard deviation
    :param dim: dimension(must be positive and also an odd number)
    :return: return the required Gaussian kernel.
    '''
    temp = [t - (dim//2) for t in range(dim)]
    assistant = []
    for i in range(dim):
        assistant.append(temp)
    assistant = np.array(assistant)
    temp = 2*sigma*sigma
    result = (1.0/(temp*np.pi))*np.exp(-(assistant**2+(assistant.T)**2)/temp)
    return result

def getDoG(img,n,sigma0,S = None,O = None): # Difference of Gaussian 정의 함수
    '''
    :파라미터 img: 기존 이미지
    :파라미터 sigma0: 첫 octave 스택에 대한 시그마 default 1.52.
    :파라미터 n: 추출하고자 하는 피처에서 몇개의 스택을 추출할 것인가
    :파라미터 S: 모든 octave가 반드시 가져야 하는 stack의 최소 개수. S는 반드시 3보다 커야 함.
    :파라미터 k: 두개의 인접 스택의 크기 비율
    :파라미터 O: 몇개의 octave를 가지고 있는지
    '''
    if S == None:
        S = n + 3
    if O == None:
        O = int(np.log2(min(img.shape[0], img.shape[1]))) - 3

    k = 2 ** (1.0 / n)
    sigma = [[(k**s)*sigma0*(1<<o) for s in range(S)] for o in range(O)]
    samplePyramid = [downsample(img, 1 << o) for o in range(O)]

    GuassianPyramid = []
    for i in range(O):
        GuassianPyramid.append([])
        for j in range(S):
            dim = int(6*sigma[i][j] + 1)
            if dim % 2 == 0:
                dim += 1
            GuassianPyramid[-1].append(convolve(GuassianKernel(sigma[i][j], dim),samplePyramid[i],[dim//2,dim//2,dim//2,dim//2],[1,1]))
    DoG = [[GuassianPyramid[o][s+1] - GuassianPyramid[o][s] for s in range(S - 1)] for o in range(O)]


    return DoG,GuassianPyramid

def adjustLocalExtrema(DoG,o,s,x,y,contrastThreshold,edgeThreshold,sigma,n,SIFT_FIXPT_SCALE): # 지역 극값을 파악하는 함수
    SIFT_MAX_INTERP_STEPS = 5
    SIFT_IMG_BORDER = 5

    point = []

    img_scale = 1.0 / (255 * SIFT_FIXPT_SCALE)
    deriv_scale = img_scale * 0.5
    second_deriv_scale = img_scale
    cross_deriv_scale = img_scale * 0.25

    img = DoG[o][s]
    i = 0
    while i < SIFT_MAX_INTERP_STEPS:
        if s < 1 or s > n or y < SIFT_IMG_BORDER or y >= img.shape[1] - SIFT_IMG_BORDER or x < SIFT_IMG_BORDER or x >= img.shape[0] - SIFT_IMG_BORDER:
            return None,None,None,None

        img = DoG[o][s]
        prev = DoG[o][s - 1]
        next = DoG[o][s + 1]

        dD = [ (img[x,y + 1] - img[x, y - 1]) * deriv_scale, # 스케일 줄여가며 
               (img[x + 1, y] - img[x - 1, y]) * deriv_scale,
               (next[x, y] - prev[x, y]) * deriv_scale ]

        v2 = img[x, y] * 2
        dxx = (img[x, y + 1] + img[x, y - 1] - v2) * second_deriv_scale
        dyy = (img[x + 1, y] + img[x - 1, y] - v2) * second_deriv_scale
        dss = (next[x, y] + prev[x, y] - v2) * second_deriv_scale
        dxy = (img[x + 1, y + 1] - img[x + 1, y - 1] - img[x - 1, y + 1] + img[x - 1, y - 1]) * cross_deriv_scale
        dxs = (next[x, y + 1] - next[x, y - 1] - prev[x, y + 1] + prev[x, y - 1]) * cross_deriv_scale
        dys = (next[x + 1, y] - next[x - 1, y] - prev[x + 1, y] + prev[x - 1, y]) * cross_deriv_scale

        H=[ [dxx, dxy, dxs],
            [dxy, dyy, dys],
            [dxs, dys, dss]]

        X = np.matmul(np.linalg.pinv(np.array(H)),np.array(dD))

        xi = -X[2]
        xr = -X[1]
        xc = -X[0]

        if np.abs(xi) < 0.5 and np.abs(xr) < 0.5 and np.abs(xc) < 0.5:
            break

        y += int(np.round(xc))
        x += int(np.round(xr))
        s += int(np.round(xi))

        i+=1

    if i >= SIFT_MAX_INTERP_STEPS:
        return None,x,y,s
    if s < 1 or s > n or y < SIFT_IMG_BORDER or y >= img.shape[1] - SIFT_IMG_BORDER or x < SIFT_IMG_BORDER or x >= \
            img.shape[0] - SIFT_IMG_BORDER:
        return None, None, None, None


    t = (np.array(dD)).dot(np.array([xc, xr, xi]))

    contr = img[x,y] * img_scale + t * 0.5
    if np.abs( contr) * n < contrastThreshold:
        return None,x,y,s


    # 헤세 행렬의 궤적과 행렬식을 사용하여 주 곡률의 비율 계산.
    tr = dxx + dyy
    det = dxx * dyy - dxy * dxy
    if det <= 0 or tr * tr * edgeThreshold >= (edgeThreshold + 1) * (edgeThreshold + 1) * det:
        return None,x,y,s

    point.append((x + xr) * (1 << o))
    point.append((y + xc) * (1 << o))
    point.append(o + (s << 8) + (int(np.round((xi + 0.5)) * 255) << 16))
    point.append(sigma * np.power(2.0, (s + xi) / n)*(1 << o) * 2)

    return point,x,y,s

def GetMainDirection(img,r,c,radius,sigma,BinNum): # keypoint의 방향 찾기
    expf_scale = -1.0 / (2.0 * sigma * sigma)

    X = []
    Y = []
    W = []
    temphist = []

    for i in range(BinNum):
        temphist.append(0.0)

    # 이미지 기울기 히스토그램 통계의 픽셀 범위
    k = 0
    for i in range(-radius,radius+1):
        y = r + i
        if y <= 0 or y >= img.shape[0] - 1:
            continue
        for j in range(-radius,radius+1):
            x = c + j
            if x <= 0 or x >= img.shape[1] - 1:
                continue

            dx = (img[y, x + 1] - img[y, x - 1])
            dy = (img[y - 1, x] - img[y + 1, x])

            X.append(dx)
            Y.append(dy)
            W.append((i * i + j * j) * expf_scale)
            k += 1


    length = k

    W = np.exp(np.array(W))
    Y = np.array(Y)
    X = np.array(X)
    Ori = np.arctan2(Y,X)*180/np.pi
    Mag = (X**2+Y**2)**0.5

    # 히스토그램의 각 bin값 계산
    for k in range(length):
        bin = int(np.round((BinNum / 360.0) * Ori[k]))
        if bin >= BinNum:
            bin -= BinNum
        if bin < 0:
            bin += BinNum
        temphist[bin] += W[k] * Mag[k]

    # smooth the histogram
    # 가우스 평활화
    temp = [temphist[BinNum - 1], temphist[BinNum - 2], temphist[0], temphist[1]]
    temphist.insert(0, temp[0])
    temphist.insert(0, temp[1])
    temphist.insert(len(temphist), temp[2])
    temphist.insert(len(temphist), temp[3])      # 패딩 진행

    hist = []
    for i in range(BinNum):
        hist.append((temphist[i] + temphist[i+4]) * (1.0 / 16.0) + (temphist[i+1] + temphist[i+3]) * (4.0 / 16.0) + temphist[i+2] * (6.0 / 16.0))

    # keypoint가 갖는 최종 방향 추출
    maxval = max(hist)

    return maxval,hist

def LocateKeyPoint(DoG,sigma,GuassianPyramid,n,BinNum = 36,contrastThreshold = 0.04,edgeThreshold = 10.0): # keypoint 위치 추출
    SIFT_ORI_SIG_FCTR = 1.52
    SIFT_ORI_RADIUS = 3 * SIFT_ORI_SIG_FCTR
    SIFT_ORI_PEAK_RATIO = 0.8

    SIFT_INT_DESCR_FCTR = 512.0
    # SIFT_FIXPT_SCALE = 48
    SIFT_FIXPT_SCALE = 1

    KeyPoints = []
    O = len(DoG)
    S = len(DoG[0])
    for o in range(O):
        for s in range(1,S-1):
            threshold = 0.5*contrastThreshold/(n*255*SIFT_FIXPT_SCALE) # 스케일 줄여가며
            img_prev = DoG[o][s-1]
            img = DoG[o][s]
            img_next = DoG[o][s+1]
            for i in range(img.shape[0]):
                for j in range(img.shape[1]):
                    val = img[i,j]
                    eight_neiborhood_prev = img_prev[max(0, i - 1):min(i + 2, img_prev.shape[0]), max(0, j - 1):min(j + 2, img_prev.shape[1])] 
                    eight_neiborhood = img[max(0, i - 1):min(i + 2, img.shape[0]), max(0, j - 1):min(j + 2, img.shape[1])]
                    eight_neiborhood_next = img_next[max(0, i - 1):min(i + 2, img_next.shape[0]), max(0, j - 1):min(j + 2, img_next.shape[1])]
                    if np.abs(val) > threshold and \
                        ((val > 0 and (val >= eight_neiborhood_prev).all() and (val >= eight_neiborhood).all() and (val >= eight_neiborhood_next).all())
                         or (val < 0 and (val <= eight_neiborhood_prev).all() and (val <= eight_neiborhood).all() and (val <= eight_neiborhood_next).all())):

                        point,x,y,layer = adjustLocalExtrema(DoG,o,s,i,j,contrastThreshold,edgeThreshold,sigma,n,SIFT_FIXPT_SCALE)
                        if point == None:
                            continue

                        scl_octv = point[-1]*0.5/(1 << o)
                        omax,hist = GetMainDirection(GuassianPyramid[o][layer],x,y,int(np.round(SIFT_ORI_RADIUS * scl_octv)),SIFT_ORI_SIG_FCTR * scl_octv,BinNum)
                        mag_thr = omax * SIFT_ORI_PEAK_RATIO
                        for k in range(BinNum):
                            if k > 0:
                                l = k - 1
                            else:
                                l = BinNum - 1
                            if k < BinNum - 1:
                                r2 = k + 1
                            else:
                                r2 = 0
                            if hist[k] > hist[l] and hist[k] > hist[r2] and hist[k] >= mag_thr:
                                bin = k + 0.5 * (hist[l]-hist[r2]) /(hist[l] - 2 * hist[k] + hist[r2])
                                if bin < 0:
                                    bin = BinNum + bin
                                else:
                                    if bin >= BinNum:
                                        bin = bin - BinNum
                                temp = point[:]
                                temp.append((360.0/BinNum) * bin)
                                KeyPoints.append(temp)


    return KeyPoints


def calcSIFTDescriptor(img,ptf,ori,scl,d,n,SIFT_DESCR_SCL_FCTR = 3.0,SIFT_DESCR_MAG_THR = 0.2,SIFT_INT_DESCR_FCTR = 512.0,FLT_EPSILON = 1.19209290E-07):
    # SIFT 서술자 계산 함수
    dst = []
    pt = [int(np.round(ptf[0])), int(np.round(ptf[1]))] # 좌표가 소수로 나올 경우 반올림
    cos_t = np.cos(ori * (np.pi / 180)) # 코사인
    sin_t = np.sin(ori * (np.pi / 180)) # 사인함수
    bins_per_rad = n / 360.0
    exp_scale = -1.0 / (d * d * 0.5)
    hist_width = SIFT_DESCR_SCL_FCTR * scl
    radius = int(np.round(hist_width * 1.4142135623730951 * (d + 1) * 0.5))
    cos_t /= hist_width
    sin_t /= hist_width

    rows = img.shape[0]
    cols = img.shape[1]


    hist = [0.0]*((d+2)*(d+2)*(n+2))
    X = []
    Y = []
    RBin = []
    CBin = []
    W = []

    k = 0
    for i in range(-radius,radius+1):
        for j in range(-radius,radius+1):

            c_rot = j * cos_t - i * sin_t
            r_rot = j * sin_t + i * cos_t
            rbin = r_rot + d // 2 - 0.5
            cbin = c_rot + d // 2 - 0.5
            r = pt[1] + i
            c = pt[0] + j

            if rbin > -1 and rbin < d and cbin > -1 and cbin < d and r > 0 and r < rows - 1 and c > 0 and c < cols - 1:
                dx = (img[r, c+1] - img[r, c-1])
                dy = (img[r-1, c] - img[r+1, c])
                X.append(dx)
                Y.append(dy)
                RBin.append(rbin)
                CBin.append(cbin)
                W.append((c_rot * c_rot + r_rot * r_rot) * exp_scale)
                k+=1

    length = k
    Y = np.array(Y)
    X = np.array(X)
    Ori = np.arctan2(Y,X)*180/np.pi
    Mag = (X ** 2 + Y ** 2) ** 0.5
    W = np.exp(np.array(W))

    for k in range(length):
        rbin = RBin[k]
        cbin = CBin[k]
        obin = (Ori[k] - ori) * bins_per_rad
        mag = Mag[k] * W[k]

        r0 = int(rbin)
        c0 = int(cbin)
        o0 = int(obin)
        rbin -= r0
        cbin -= c0
        obin -= o0

        if o0 < 0:
            o0 += n
        if o0 >= n:
            o0 -= n

        # 삼선형 보간법을 사용한 히스토그램 업데이트
        v_r1 = mag * rbin
        v_r0 = mag - v_r1

        v_rc11 = v_r1 * cbin
        v_rc10 = v_r1 - v_rc11

        v_rc01 = v_r0 * cbin
        v_rc00 = v_r0 - v_rc01

        v_rco111 = v_rc11 * obin
        v_rco110 = v_rc11 - v_rco111

        v_rco101 = v_rc10 * obin
        v_rco100 = v_rc10 - v_rco101

        v_rco011 = v_rc01 * obin
        v_rco010 = v_rc01 - v_rco011

        v_rco001 = v_rc00 * obin
        v_rco000 = v_rc00 - v_rco001

        idx = ((r0 + 1) * (d + 2) + c0 + 1) * (n + 2) + o0
        hist[idx] += v_rco000
        hist[idx+1] += v_rco001
        hist[idx + (n+2)] += v_rco010
        hist[idx + (n+3)] += v_rco011
        hist[idx+(d+2) * (n+2)] += v_rco100
        hist[idx+(d+2) * (n+2)+1] += v_rco101
        hist[idx+(d+3) * (n+2)] += v_rco110
        hist[idx+(d+3) * (n+2)+1] += v_rco111

    # 방향 히스토그램이 원형이므로 히스토그램을 완료
    for i in range(d):
        for j in range(d):
            idx = ((i+1) * (d+2) + (j+1)) * (n+2)
            hist[idx] += hist[idx+n]
            hist[idx+1] += hist[idx+n+1]
            for k in range(n):
                dst.append(hist[idx+k])


    # 히스토그램을 descriptor에 복사 후
    # 히스테리시스 임계값 적용
    # 쉽게 변환할 수 있도록 결과 크기를 조정
    # 바이트 배열로

    nrm2 = 0
    length = d * d * n
    for k in range(length):
        nrm2 += dst[k] * dst[k]
    thr = np.sqrt(nrm2) * SIFT_DESCR_MAG_THR

    nrm2 = 0
    for i in range(length):
        val = min(dst[i], thr)
        dst[i] = val
        nrm2 += val * val
    nrm2 = SIFT_INT_DESCR_FCTR / max(np.sqrt(nrm2), FLT_EPSILON)
    for k in range(length):
        dst[k] = min(max(dst[k] * nrm2,0),255)

    return dst


def calcDescriptors(gpyr,keypoints,SIFT_DESCR_WIDTH = 4,SIFT_DESCR_HIST_BINS = 8):
    # SIFT_DESCR_WIDTH = 4，히스토그램의 너비
    # SIFT_DESCR_HIST_BINS = 8
    d = SIFT_DESCR_WIDTH
    n = SIFT_DESCR_HIST_BINS
    descriptors = []

    for i in range(len(keypoints)):
        kpt = keypoints[i]
        o = kpt[2] & 255
        s = (kpt[2] >> 8) & 255 # keypoint 그룹 번호 및 레이어 번호
        scale = 1.0 / (1 << o)  # 스케일링 범위
        size = kpt[3] * scale # keypoint 그룹의 이미지 크기
        ptf = [kpt[1] * scale, kpt[0] * scale] # 피라미드 그룹의 keypoint 좌표
        img = gpyr[o][s] # keypoint가 위치한 피라미드의 이미지

        descriptors.append(calcSIFTDescriptor(img, ptf, kpt[-1], size * 0.5, d, n))
    return descriptors

def SIFT(img,showDoGimgs = False):
    SIFT_SIGMA = 1.6
    SIFT_INIT_SIGMA = 0.5 
    sigma0 = np.sqrt(SIFT_SIGMA**2-SIFT_INIT_SIGMA**2)

    n = 3

    DoG,GuassianPyramid = getDoG(img, n,sigma0)
    if showDoGimgs:
        for i in DoG:
            for j in i:
                plt.imshow(j.astype(np.uint8), cmap='gray')
                plt.axis('off')
                plt.show()

    KeyPoints = LocateKeyPoint(DoG, SIFT_SIGMA, GuassianPyramid, n)
    discriptors = calcDescriptors(GuassianPyramid,KeyPoints)

    return KeyPoints,discriptors


def Lines(img,info,color = (255,0,0),err = 700):

    if len(img.shape) == 2:
        result = np.dstack((img,img,img))
    else:
        result = img
    k = 0
    for i in range(result.shape[0]):
        for j in range(result.shape[1]):
            temp = (info[:,1]-info[:,0])
            A = (j - info[:,0])*(info[:,3]-info[:,2])
            B = (i - info[:,2])*(info[:,1]-info[:,0])
            temp[temp == 0] = 1e-9
            t = (j-info[:,0])/temp
            e = np.abs(A-B)
            temp = e < err
            if (temp*(t >= 0)*(t <= 1)).any():
                result[i,j] = color
                k+=1
    print(k)

    return result

def drawLines(X1,X2,Y1,Y2,dis,img,num = 10):

    info = list(np.dstack((X1,X2,Y1,Y2,dis))[0])
    info = sorted(info,key=lambda x:x[-1])
    info = np.array(info)
    info = info[:min(num,info.shape[0]),:]
    img = Lines(img,info)

    if len(img.shape) == 2:
        plt.imshow(img.astype(np.uint8),cmap='gray')
    else:
        plt.imshow(img.astype(np.uint8))
    plt.axis('off')
    plt.show()


if __name__ == '__main__':
    origimg = plt.imread('home.jpeg')
    if len(origimg.shape) ==  3:
        img = origimg.mean(axis=-1)
    else:
        img = origimg
    keyPoints,discriptors = SIFT(img)


    origimg2 = plt.imread('tmp.jpeg')
    if len(origimg.shape) == 3:
        img2 = origimg2.mean(axis=-1)
    else:
        img2 = origimg2
    ScaleRatio = img.shape[0]*1.0/img2.shape[0]

    img2 = np.array(Image.fromarray(img2).resize((int(round(ScaleRatio * img2.shape[1])),img.shape[0]), Image.BICUBIC))
    keyPoints2, discriptors2 = SIFT(img2,True)

    knn = KNeighborsClassifier(n_neighbors=1)
    knn.fit(discriptors,[0]*len(discriptors))
    match = knn.kneighbors(discriptors2,n_neighbors=1,return_distance=True)

    keyPoints = np.array(keyPoints)[:,:2]
    keyPoints2 = np.array(keyPoints2)[:,:2]

    keyPoints2[:, 1] = img.shape[1] + keyPoints2[:, 1]

    origimg2 = np.array(Image.fromarray(origimg2).resize((img2.shape[1],img2.shape[0]), Image.BICUBIC))
    result = np.hstack((origimg,origimg2))


    keyPoints = keyPoints[match[1][:,0]]

    X1 = keyPoints[:, 1]
    X2 = keyPoints2[:, 1]
    Y1 = keyPoints[:, 0]
    Y2 = keyPoints2[:, 0]

    drawLines(X1,X2,Y1,Y2,match[0][:,0],result)



# SURF

In [ ]:
!pip uninstall opencv-contrib-python
!pip uninstall opencv-python

!pip install --user opencv-python==3.4.2.17 opencv-contrib-python==3.4.2.17

In [ ]:
# Introduction to SURF (Speeded-Up Robust Features)
# SIFT에 비해 속도 개선됨
# blurry한 이미지, 이미지 회전에 관계없이 feature 추출 가능

img = cv.imread('home.jpg',0)
surf = cv.xfeatures2d.SURF_create(400)
kp, des = surf.detectAndCompute(img,None)
print(len(kp))

In [ ]:
print( surf.getHessianThreshold() )
surf.setHessianThreshold(50000)
kp, des = surf.detectAndCompute(img,None)
print( len(kp) )

In [ ]:
img2 = cv.drawKeypoints(img,kp,None,(255,0,0),4)
plt.imshow(img2),plt.show()

In [ ]:
kp = surf.detect(img,None)
img2 = cv.drawKeypoints(img,kp,None,(255,0,0),4)
plt.imshow(img2),plt.show()

In [ ]:
### SURF scratch 코드 파일을 찾을 수 없어 다른 형식으로 사용해본 코드를 분석해보았습니다.

import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt


data = pd.read_csv(r'.\sign data\data.csv')


# 각 클래스 샘플의 개수


le = [242, 259, 247, 147, 243, 226, 241, 116, 179, 245, 182, 235, 237, 229, 234, 216, 211, 229, 216, 135, 122, 128, 202, 251]

label = []
for ini,folder in enumerate(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']):
    i = ord(folder)
    temp = [i-65]*le[ini]
    label.append(temp)



# 레이블을 numpy array로 변환

label = np.concatenate(label) 


# 레이블 딕셔너리 클래스 생성

ans = {}
for i in range(65,90):
    ans[i-65] = chr(i)
del ans[9]

x, y = data.values, label
# ind = 1090
# plt.imshow(x[ind].reshape(-1,128),cmap='gray')
# print(ans[y[ind]])
# plt.show()


img = cv2.GaussianBlur(x[87].reshape(-1,128).astype(np.uint8),(5,5),0)
edge = cv2.Canny(img,60,60)
plt.imshow(edge,cmap='gray')
plt.show()


surf = cv2.xfeatures2d.SURF_create()
kp = surf.detect(edge,None)
kp, des = surf.compute(edge, kp)
img2 = cv2.drawKeypoints(edge,kp,color=(0,255,0),outImage=None, flags=0)
plt.imshow(img2),plt.show()
print(des.shape[0]) 




def pre_processing(img):
    # canny edge detector에는 노이즈 감소, 비최대 억제 등이 포함 -> 이러한 장점을 사용
    edge = cv2.Canny(img,65,65)
    return edge



# 각 이미지를 이동하고 해당 이미지와 관련된 기능 추가

surf = cv2.xfeatures2d.SURF_create()
temp = []
for i in range(x.shape[0]):
    edge = pre_processing(x[i].reshape(-1,128).astype(np.uint8))
    kp, des = surf.detectAndCompute(edge, None)
    temp.append(des)


# 모든 이미지의 모든 feature(128개 요소 벡터) 추가
train_desc = []
for desc_list in temp:
    for desc in desc_list:
        train_desc.append(desc)


train_desc = np.array(train_desc)


# k-means 클러스터링을 사용하여 기능을 클러스터링하여 시각적 단어 모델의 vbag 생성

n_clusters = 125
from sklearn.cluster import MiniBatchKMeans
cls = MiniBatchKMeans(n_clusters)


cls.fit(train_desc)


# DBSCAN에서 찾은 클러스터 중심을 사용하여 image feature에 대한 히스토그램 제작
pred = []
for desc_list in temp:
    pred.append(cls.predict(desc_list))

# 속해 있는 feature의 예측 클러스터에서 이미지에 대한 feature 벡터를 생성.
train_hist = [np.bincount(feature,minlength=n_clusters) for feature in pred]


train_hist = np.array(train_hist)



from sklearn.svm import SVC 
clf = SVC(kernel='linear')

 
from sklearn.utils import shuffle
a, b = shuffle(train_hist, label, random_state=25)


# 일정 범위의 값으로 정규화

p = a.max(axis=0)
a_ = []
for i in range(a.shape[0]):
    a_.append(a[i]/(p*1.0))

a_ = np.array(a_)

# train validation 분학

x_train, y_train, x_test, y_test = a_[:4400], b[:4400], a_[4400:], b[4400:]


clf.fit(x_train,y_train)

print(clf.score(x_train,y_train)*100)



print(clf.score(x_test,y_test)*100)



from sklearn.ensemble import RandomForestClassifier


clf1 = RandomForestClassifier(n_estimators = 50,n_jobs=-1)



clf1.fit(x_train,y_train)

print(clf1.score(x_train,y_train)*100)


print(clf1.score(x_test,y_test)*100)



from sklearn.neural_network import MLPClassifier
clf3 = MLPClassifier()

clf3.fit(x_train,y_train)


print(clf1.score(x_train,y_train)*100)


print(clf1.score(x_test,y_test)*100)


# SURF는 모든 기능 추출기 중에서 가장 잘 수행되는 경향이 있으므로 동일한 항목에서 혼동 행렬을 시각화 가능.
x = clf.predict(x_val)

from sklearn.metrics import confusion_matrix
m = confusion_matrix(y_val,x)

import seaborn as sns
df_cm = pd.DataFrame(m, ans.values(), ans.values())
plt.figure(figsize = (15,10))
sns.heatmap(df_cm, annot=True,fmt='g')

# FAST

In [ ]:
### FAST Algorithm for Corner Detection
# SURF : SIFT보다 빠르나 real - time에는 어려움 있음 -> FAST 등장
# 실시간으로 물체의 특징을 감지하기 위해 ... 라이다(Lidar)

img = cv.imread('chessboard2.jpg',0)
# Initiate FAST object with default values
fast = cv.FastFeatureDetector_create()
# find and draw the keypoints
kp = fast.detect(img,None)
img2 = cv.drawKeypoints(img, kp, None, color=(255,0,0))
# Print all default params
print( "Threshold: {}".format(fast.getThreshold()) )
print( "nonmaxSuppression:{}".format(fast.getNonmaxSuppression()) )
print( "neighborhood: {}".format(fast.getType()) )
print( "Total Keypoints with nonmaxSuppression: {}".format(len(kp)) )
cv.imwrite('fast_true.png', img2)
# Disable nonmaxSuppression
fast.setNonmaxSuppression(0)
kp = fast.detect(img, None)
print( "Total Keypoints without nonmaxSuppression: {}".format(len(kp)) )
img3 = cv.drawKeypoints(img, kp, None, color=(255,0,0))

cv2_imshow(img3)

In [ ]:
"""
Performs FAST corner detection without machine generated code.

Reference:
    - http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_feature2d/py_fast/py_fast.html
"""


""" 
***** Begin NumPy and OpenCV functions. ******
"""

def shape(array):
    """ 
    2차원 array의 shape 출력
    """
    rows = len(array)

    cols = len(array[0])
    return [rows, cols]

def zeros(rows, cols):
    """
    모두 0으로 채워진 2차원 array 출력
    """
    return [[0 for col in range(cols)] for row in range(rows)]

def rgb2gray(array):
    """
    흑백 변환
    """
    rows, cols = shape(array)
    grayscale = zeros(rows, cols)
    for row in range(rows):
        for col in range(cols):
            red, green, blue = array[row][col]
            gray = int(0.3*red + 0.59*green + 0.11*blue)
            grayscale[row][col] = gray
    return grayscale

def medianBlur(image, startSearchRow, endSearchRow, startSearchCol, endSearchCol, N=3):
    """
    이미지에 Median blur 효과를 수행
    각 픽셀을 주변 NxN 픽셀의 중앙값으로
    N은 홀수 정수여야 함
    검색 영역에 대해서만 Median blur 진행
    """
    dst = image[:] 
    for y in range(startSearchRow, endSearchRow):
        for x in range(startSearchCol, endSearchCol):
            window = []
            for i in range(y - N // 2, y + N // 2 + 1):
                for j in range(x - N//2, x + N//2 + 1):
                    window.append(image[i][j])
            insertionSort(window)
            dst[y][x] = window[len(window)//2]

    return dst
    
def insertionSort(lst):
    for index in range(1, len(lst)):
        currentvalue = lst[index]
        position = index

        while position > 0 and lst[position - 1] > currentvalue:
            lst[position] = lst[position - 1]
            position = position - 1

        lst[position] = currentvalue

def circle(row, col):
    """ 
    픽셀 검색 영역의 둘레를 구성하는 일부 픽셀((x,y) 튜플)의 목록을 반환
    Circle circumference = 16 픽셀

    """
    point1 = (row+3, col)
    
    point3 = (row+3, col-1)
    
    point5 = (row+1, col+3)
    
    point7 = (row-1, col+3)
    
    point9 = (row-3, col)
    
    point11 = (row-3, col-1)
    
    point13 = (row+1, col-3)
    
    point15 = (row-1, col-3)
    
    return [point1, point3, point5, point7, point9, point11, point13, point15];

def is_corner(image, row, col, ROI, threshold):
    """
    circle함수에서 반환된 것과 동일한 픽셀을 사용
   
    Method:
        픽셀 1의 강도가 임계값 기준을 충족하는 경우 픽셀 3과 15도 임계값 기준을 충족하는지 확인 -> 조건 충족 시 corner
        circle 함수에서 반환된 모든 점에 대해 반복
        어떤 기준도 충족되지 않으면 -> corner 아님
        
        12개의 연속 픽셀 방법을 따라 여러 점을 확인

    임계값을 높은 값으로 설정하여 더 많은 비 모서리를 걸러낼 수 있음
    """
    intensity = int(image[row][col])
    row1, col1 = ROI[0]
    row9, col9 = ROI[4]
    row5, col5 = ROI[2]
    row13, col13 = ROI[6]
    intensity1 = int(image[row1][col1])
    intensity9 = int(image[row9][col9])
    intensity5 = int(image[row5][col5])
    intensity13 = int(image[row13][col13])
    count = 0
    if abs(intensity1 - intensity) > threshold:
        count += 1 
    if abs(intensity9 - intensity) > threshold:
        count += 1
    if abs(intensity5 - intensity) > threshold:
        count += 1
    if abs(intensity13 - intensity) > threshold:
        count += 1

    return count >= 3

def areAdjacent(point1, point2):
    """
    행/열로 거리를 계산하여 두 점이 인접하는지 식별
    두 점이 서로 4픽셀 이내에 있으면 인접 (유클리디안 거리)
    """
    row1, col1 = point1
    row2, col2 = point2
    xDist = row1 - row2
    yDist = col1 - col2
    return (xDist ** 2 + yDist ** 2) ** 0.5 <= 4

def calculateScore(image, point, ROI):
    """ 
    non-maximal suppression 점수 계산
    점수 V는 강도 간의 절댓값 차이의 합으로 정의
    circle 함수에 의해 반환된 모든 점과 중심 픽셀의 강도
    """
    col, row = point
    intensity = int(image[row][col])
    row1, col1 = ROI[0]
    row3, col3 = ROI[1]
    row5, col5 = ROI[2]
    row7, col7 = ROI[3]
    row9, col9 = ROI[4]
    row11, col11 = ROI[5]
    row13, col13 = ROI[6]
    row15, col15 = ROI[7]
    intensity1 = int(image[row1][col1])
    intensity3 = int(image[row3][col3])
    intensity5 = int(image[row5][col5])
    intensity7 = int(image[row7][col7])
    intensity9 = int(image[row9][col9])
    intensity11 = int(image[row11][col11])
    intensity13 = int(image[row13][col13])
    intensity15 = int(image[row15][col15])   
    score = abs(intensity - intensity1) + abs(intensity - intensity3) + \
            abs(intensity - intensity5) + abs(intensity - intensity7) + \
            abs(intensity - intensity9) + abs(intensity - intensity11) + \
            abs(intensity - intensity13) + abs(intensity - intensity15)
    return score

def suppress(image, corners, ROI):
    """
    list of corners 에서 non-maximal suppression 수행
    인접한 모서리의 경우 점수가 가장 작은 모서리 배제
    그렇지 않은경우 아무것도 하지 않음

    이미지의 모든 픽셀을 순서대로 반복하기 때문에 인접한 corner point는 모든 list of corners에서 서로 옆에 붙어 있어야 함
    """
    i = 1
    while i < len(corners):
        currPoint = corners[i]
        prevPoint = corners[i - 1]
        if areAdjacent(prevPoint, currPoint):
            currScore = calculateScore(image, currPoint, ROI)
            prevScore = calculateScore(image, prevPoint, ROI)
            if (currScore > prevScore):
                del(corners[i - 1])
            else:
                del(corners[i])
        else:
            i += 1
            continue
    return



def detect(image, threshold=50):
    """
    이미지에서 코너에 해당하는 픽셀을 (x,y) 튜플로 반환함

    기본적으로 non-maximal suppression 수행

    이미지 전체에서 코너 검색을 하는 것이 아니라 프로세스 속도를 높이기 위해 중간 부분만 검색

    이미지는 반드시 흑백 사진이어야 함
    """
    # Initialization
    image = rgb2gray(image)
    corners = []
    rows,cols = shape(image)
    startSearchRow = int(0.25*rows)
    endSearchRow = int(0.75*rows) # search the middle square of the frame
    startSearchCol = int(0.25*cols)
    endSearchCol = int(0.75*cols)
    image = medianBlur(image, startSearchRow, endSearchRow, startSearchCol, endSearchCol)

    # Begin searching through search area
    for row in range(startSearchRow, endSearchRow):
        for col in range(startSearchCol, endSearchCol):
            ROI = circle(row, col) 
            if is_corner(image, row, col, ROI, threshold):
                corners.append((col, row))
    suppress(image, corners, ROI) 
    return corners;

In [ ]:
detect('home.jpg')